In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re
import os
import shutil
import smart_open
from sys import platform
import gensim


In [4]:
gensim_file='glove_model2.txt'
model=gensim.models.KeyedVectors.load_word2vec_format(gensim_file,binary=False) #GloVe Model


In [5]:
exp = "[a-zA-Z0-9]+"

In [6]:
def vector_avg(vec_list):
    if len(vec_list)!=0:
        return sum(vec_list)/len(vec_list)
    else:
        return np.zeros(300)

    #return np.mean(vec_list)


def get_vector(med):   #"kirk vit"
    split_med=med.split()   #["kirk","vit"]
    
    vec_list=[]
    for item in split_med:
        try:
            vec_list.append(model[item])  #converting each word in the medicine name to a 
        except:
            #print("exception!!")
            continue
        
        
        '''if model[item]:
            vec_list.append(model[item])  #converting each word in the medicine name to a 
        else:
            continue'''
        
    #calc vector avg of all words in medicine name    
    vector=vector_avg(vec_list)    
        
    return vector
    
    
def get_similarity(vector):
    sim=model.similar_by_vector(vector, topn=11, restrict_vocab=None)
    return sim
    

In [7]:
def medstring_to_vector(medstring):
    meds=re.findall(exp,medstring)
    #print(meds)    #['kirkland multivitamin', 'kirkland calcium vitamin', 'vitamin d', 'fish oil']

    #list of vectors of the medicine names in each row
    #[v(kirkland multivitamin), v(kirkland calcium vitamin), v(vitamin d), v(fish oil)]
    vec_list=[]  

    for item in meds:
        #try:             
        v=get_vector(item)
        vec_list.append(v)                            

    #avg of all med vectors in a row (for a user/entry)
    if vec_list:
        meds_avg=vector_avg(vec_list)
        return meds_avg
    else:
        return -1

In [8]:
def add_vector_column(df): 
    df["meds_vector"] = ""
    df['allergies_vector']=""
    df['history_vector']=""

    for i,j in df["other_meds_filtered"].iteritems():
        #j = "['kirkland multivitamin', 'kirkland calcium vitamin', 'vitamin d', 'fish oil']"

        #if not pd.isna(j):
        if not pd.isnull(j):
            meds_avg = medstring_to_vector(j)
            if type(meds_avg) != int:
                df["meds_vector"].loc[i]= meds_avg
                
    for i,j in df["allergies_filtered"].iteritems():
              #if not pd.isna(j):
        if not pd.isnull(j):
            aller_avg = medstring_to_vector(j)
            if type(aller_avg) != int:
                df["allergies_vector"].loc[i]= aller_avg
                
    for i,j in df["history_filtered"].iteritems():
              #if not pd.isna(j):
        if not pd.isnull(j):
            his_avg = medstring_to_vector(j)
            if type(his_avg) != int:
                df["history_vector"].loc[i]= aller_avg

In [9]:
data = pd.read_csv(r'someeee.csv')
data.head()

C:\Users\santosh bishnoi\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (8,13,16,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,...,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,other_meds_filtered,allergies_filtered,history_filtered,meds_vector
0,0,916600,01-01-2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,...,2,01-01-2021,NaN,Y,NaN,Pcn and bee venom,NaN,"['pcn', 'bee venom']",NaN,NaN
1,1,916601,01-01-2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,...,2,01-01-2021,NaN,Y,NaN,"""Dairy""",['residing nursing facility . patients chart .'],"[""`` dairy ''""]",['residing nursing facility . patients chart .'],NaN
2,2,916602,01-01-2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",...,2,01-01-2021,NaN,NaN,Y,Shellfish,NaN,['shellfish'],NaN,NaN
3,3,916603,01-01-2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",...,2,01-01-2021,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom...",NaN,"['diclofenac', 'novacaine', 'lidocaine', 'pick...","['diverticulitis', 'mitral valve prolapse', 'o...",NaN
4,4,916604,01-01-2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",...,2,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
add_vector_column(data)

C:\Users\santosh bishnoi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [11]:
data.to_csv('uncleaned_vectors.csv')

In [12]:
data.head()

,Unnamed: 0,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,...,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,other_meds_filtered,allergies_filtered,history_filtered,meds_vector,allergies_vector,history_vector
0,0,916600,01-01-2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,...,NaN,Y,NaN,Pcn and bee venom,NaN,"['pcn', 'bee venom']",NaN,,"[0.183719, -0.30023667, 0.034009665, -0.050424...",
1,1,916601,01-01-2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,...,NaN,Y,NaN,"""Dairy""",['residing nursing facility . patients chart .'],"[""`` dairy ''""]",['residing nursing facility . patients chart .'],"[0.025324296, 0.024884004, -0.2335778, 0.01385...","[-0.30221, 0.0031568, -0.14152, 0.025837, 0.59...","[0.4257, -0.52978, 0.37835, -0.28299, -0.48079..."
2,2,916602,01-01-2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",...,NaN,NaN,Y,Shellfish,NaN,['shellfish'],NaN,,"[-0.30517, -0.41038, 0.059648, -0.48996, 0.314...",
3,3,916603,01-01-2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",...,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom...",NaN,"['diclofenac', 'novacaine', 'lidocaine', 'pick...","['diverticulitis', 'mitral valve prolapse', 'o...",,"[-0.33855534, -0.17594332, 0.25119236, 0.02458...","[0.4257, -0.52978, 0.37835, -0.28299, -0.48079..."
4,4,916604,01-01-2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,
